In [7]:
import importlib
import main
import os
importlib.reload(main)
from main import *
folder_list = [
    "dataset/images_test",
    "dataset/circled_no",
    "dataset/circled_yes",
    "dataset/unticked",
    "dataset/ticked",
    
]


def list_files_in_folder(folder_path, ext=None):
    """
    Returns a list of file names in the given folder, optionally filtering by extension.
    
    Parameters:
        folder_path (str): The path to the folder.
        ext (str, optional): The file extension to filter by (e.g., 'png'). Defaults to None.
        
    Returns:
        list: A list of file names matching the extension.
    """
    try:
        # List all files in the directory
        files = [
            f for f in os.listdir(folder_path)
            if os.path.isfile(os.path.join(folder_path, f))
        ]
        
        if ext is not None:
            # Normalize the extension to lowercase
            ext = ext.lower()
            # Filter files by extension (case-insensitive)
            files = [
                f for f in files
                if os.path.splitext(f)[1].lower() == f'.{ext}'
            ]
        
        return files
    
    except FileNotFoundError:
        print(f"The folder '{folder_path}' does not exist.")
        return []
    except PermissionError:
        print(f"Permission denied to access '{folder_path}'.")
        return []
    



threshold = 60  # Set your desired threshold


for folder in folder_list:
    image_paths = list_files_in_folder(folder,ext="PNG")

    print(folder)
    for file_name in image_paths:
        full_file_path = os.path.join(folder, file_name)

        filename = full_file_path.split("/")[-1]
        name = filename.split(".")[0]
        unique_strings = [
        # "no1_93",
        # "no1_86",
        # "yes1_2",
        # "no1_96",
        # "no1_97",
        # "yes1_79",
        # "no1_27",
        # "no1_2",
        # "no4",
        # "no1_71",
        # "yes1_60",
        # "no1_22"
    ]

        # if name  in unique_strings:
        if name  in unique_strings or 1:
        # break   

            result_payload = process_image(full_file_path, threshold=threshold, show_plots=False)
            LOGGER.info(result_payload)



            # Save the resulting images with green or red text
            output_directory = 'results'
            file_name_prefix = os.path.splitext(os.path.basename(filename))[0]  # Use the original file name as prefix
            image = cv2.imread(full_file_path)
            annotated_image_path = annotate_image(
                image=image,
                result_payload=result_payload,
                output_path=output_directory,
                file_name_prefix=file_name_prefix,
        )

        

INFO:main:

 processing image 	 dataset/images_test/no1_89.PNG
DEBUG:main:comparing against templates of not circled yes/no
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (126, 71)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: 0.033988431096076965
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (126, 71)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: 0.029956892132759094
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (126, 71)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: 0.026479389518499374
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (126, 71)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between align

dataset/images_test


DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: 0.3286750018596649
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (146, 77)
DEBUG:main:Template aligned using homography.
DEBUG:main:Similarity between aligned images: 0.10280055552721024
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (146, 77)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: 0.6441971063613892
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (135, 76)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: 0.09067336469888687
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (125, 70)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: 0.0538

dataset/circled_no


DEBUG:main:Descriptors not found. Returning original template.
DEBUG:main:Similarity between aligned images: -0.0540754534304142
DEBUG:main:Template 'template_no_3.PNG' has best match with value: 0.9445103406906128
DEBUG:main:comparing against templates of ticked yes/no
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (132, 76)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: 0.09017949551343918
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (132, 75)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: 0.04625880345702171
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (132, 76)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: 0.05211704969406128
DEBUG:main:Skipping resizing of templates.
DEBUG:ma

...

DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: -0.03263786807656288
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (131, 76)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: -0.03932784125208855
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (148, 76)
DEBUG:main:Descriptors not found. Returning original template.
DEBUG:main:Similarity between aligned images: -0.10725204646587372
DEBUG:main:Template 'template_yes.PNG' has best match with value: 0.4576556980609894
DEBUG:main:comparing against templates of ticked yes/no
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (148, 76)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: 0.22095894813537598
DEBUG:main:Skipping resizing of templates.
DEBUG:

dataset/unticked


DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (135, 76)
DEBUG:main:Descriptors not found. Returning original template.
DEBUG:main:Similarity between aligned images: -0.027668427675962448
DEBUG:main:Template 'template_no_3.PNG' has best match with value: 0.25411105155944824
DEBUG:main:comparing against templates of ticked yes/no
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (135, 76)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: 0.09845428913831711
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (135, 75)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: -0.017721910029649734
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (135, 76)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity b

dataset/ticked


DEBUG:main:Similarity between aligned images: 0.49033650755882263
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (126, 71)
DEBUG:main:Descriptors not found. Returning original template.
DEBUG:main:Similarity between aligned images: -0.0794631764292717
DEBUG:main:Template 'template_no.PNG' has best match with value: 0.5923968553543091
DEBUG:main:comparing against templates of ticked yes/no
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (126, 71)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: 0.06772501766681671
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (126, 71)
DEBUG:main:Not enough good matches found. Returning original template.
DEBUG:main:Similarity between aligned images: -0.02846919186413288
DEBUG:main:Skipping resizing of templates.
DEBUG:main:Cropping images to size: (126, 71)
DEBUG:main:Not enough good matches

In [2]:
result_payload

{'confidence': 99.99983310699463,
 'final_result': True,
 'test_performed': 'checkbox_checked',
 'all_attempts': {'not_circled': {'detected': False,
   'confidence': 16.79878681898117,
   'test': 'Text not circled using templates'},
  'checkbox_checked': {'detected': True,
   'confidence': 99.99983310699463,
   'test': 'Checkbox checked using templates'},
  'checkbox_unchecked': {'detected': True,
   'confidence': 92.17464327812195,
   'test': 'Checkbox unchecked using templates'},
  'circled': {'detected': True,
   'confidence': 80,
   'test': 'Text circled detection'},
  'tick_marks': {'detected': True,
   'confidence': 0,
   'test': 'Tick marks in checkbox area'}}}

In [3]:
# epsilon_values = [0.01, 0.02, 0.03, 0.05]

# for epsilon_multiplier in epsilon_values:
#     print(f"\nTesting with epsilon_multiplier = {epsilon_multiplier}")
#     tick_detected, detection_info = detect_checkbox_with_tick(
#         gray,
#         area_threshold=100,
#         epsilon_multiplier=epsilon_multiplier,
#         debug=True,
#         show_plots=True
#     )

In [4]:
# param_ranges = {
#     'clahe_clip_limits': [1.0, 1.5, 2.0],
#     'tile_grid_sizes': [(4, 4), (8, 8)],
#     'block_sizes': [15, 21, 31],  # Must be odd numbers
#     'C_values': [2, 3, 5],
#     'area_thresholds': [50, 100, 200]
# }
# full_file_path = "dataset/images_test/no1_93.PNG"
# explore_detection_parameters(full_file_path, param_ranges)